In [1]:
from huggingface_hub import hf_hub_download 

hf_hub_download(repo_id="ai-lab/MBD-mini", filename="ptls.tar.gz", repo_type="dataset", local_dir="/kaggle/working/")
hf_hub_download(repo_id="ai-lab/MBD-mini", filename="targets.tar.gz", repo_type="dataset", local_dir="/kaggle/working/")

ptls.tar.gz:   0%|          | 0.00/1.40G [00:00<?, ?B/s]

targets.tar.gz:   0%|          | 0.00/7.93M [00:00<?, ?B/s]

'/kaggle/working/targets.tar.gz'

In [2]:
!pip install lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 14.7 MB/s eta 0:00:00a 0:00:01


In [3]:
import wandb

wandb.login(key="79f2120f8d4212aceb2c60b3c89a1b6727c19cff")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: toly-kiri4enko (bstu). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
!pip install pyspark
!pip install pytorch-lifestream

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.4/163.4 kB 3.6 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 8.3 MB/s eta 0:00:00
  Created wheel for pytorch-lifestream: filename=pytorch_lifestream-0.6.0-py3-none-any.whl size=274640 sha256=a779485ba87ed39c0d0c66fd7eb61110c0be3280160356c271707a3049d4058b
  Stored in directory: /root/.cache/pip/wheels/90/76/b4/0a944bc7c5a69201e4d757cc54886971117a2a581740e7f11d
Successfully built pytorch-lifestream


In [5]:
!tar -xf ptls.tar.gz
!tar -xf targets.tar.gz

In [6]:
import pandas as pd
import numpy as np
import os

import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import types as T
import time
import datetime
from ptls.data_load.datasets import ParquetDataset, ParquetFiles
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, ArrayType
from tqdm.notebook import tqdm
from ptls.preprocessing import PysparkDataPreprocessor
import pytorch_lightning as pl
from ptls.data_load.datasets import MemoryMapDataset
from ptls.data_load.iterable_processing import SeqLenFilter, FeatureFilter
from ptls.data_load.iterable_processing.iterable_seq_len_limit import ISeqLenLimit
from ptls.data_load.iterable_processing.to_torch_tensor import ToTorch
from ptls.frames.coles import CoLESModule
from ptls.frames import PtlsDataModule
from ptls.frames.coles import ColesDataset
from ptls.frames.coles.split_strategy import SampleSlices
import torch
import numpy as np
import pandas as pd
import calendar
from glob import glob
from ptls.data_load.utils import collate_feature_dict

from ptls.data_load.iterable_processing_dataset import IterableProcessingDataset
from datetime import datetime
from ptls.data_load.padded_batch import PaddedBatch

In [7]:
spark_conf = pyspark.SparkConf()
spark_conf.setMaster("local[*]").setAppName("JoinModality")
spark_conf.set("spark.driver.maxResultSize", "16g")
spark_conf.set("spark.executor.memory", "32g")
spark_conf.set("spark.executor.memoryOverhead", "16g")
spark_conf.set("spark.driver.memory", "32g")
spark_conf.set("spark.driver.memoryOverhead", "16g")
spark_conf.set("spark.cores.max", "8")
spark_conf.set("spark.sql.shuffle.partitions", "200")
spark_conf.set("spark.local.dir", "../../spark_local_dir")


spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()
spark.sparkContext.getConf().getAll()

[('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/jdk.internal.ref=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false'),
 ('spark.executor.memoryOverhead', '16g'),
 ('spark.driver.memory', '32g'),
 ('spark.driver.maxResultSi

In [8]:
spark

In [9]:
!mkdir /kaggle/working/mm_dataset

In [10]:
TRX_DATA_PATH = '/kaggle/working/ptls/trx/'
GEO_DATA_PATH = '/kaggle/working/ptls/geo/'
DIAL_DATA_PATH = '/kaggle/working/ptls/dialog/'

MM_DATA_PATH = '/kaggle/working/mm_dataset'
MMT_DATA_PATH = '/kaggle/working/mm_dataset_supervised'

TARGETS_DATA_PATH = '/kaggle/working/targets/'

In [11]:
def rename_col(df, prefix, col_id='client_id'):
    new_column_names = [f"{prefix}_{col}" for col in df.columns if col != col_id]
    old_column_names = [col for col in df.columns if col != col_id]
    for old_col, new_col in zip(old_column_names, new_column_names):
        df = df.withColumnRenamed(old_col, new_col)
    return df

In [12]:
from ptls.preprocessing import PysparkDataPreprocessor
from pyspark.sql.functions import explode, col

# preprocessor_dial = PysparkDataPreprocessor(
#     col_id='client_id',
#     col_event_time='event_time',
#     cols_identity=['embedding']
# )

# preprocessor_trx = PysparkDataPreprocessor(
#     col_id='client_id',
#     col_event_time='event_time',
#     cols_category=[
#         'event_type',
#         'event_subtype',
#         'src_type11',
#         'src_type12',
#         'dst_type11',
#         'dst_type12',
#         'src_type21',
#         'src_type22',
#         'src_type31',
#         'src_type32',
#     ],
#     cols_identity=['amount', 'currency']
# )


for fold in tqdm(range(0, 5)):
    trx = spark.read.parquet(os.path.join(TRX_DATA_PATH, f'fold={fold}'))
    dial = spark.read.parquet(os.path.join(DIAL_DATA_PATH, f'fold={fold}'))
    
    trx = rename_col(trx, 'trx')
    dial = rename_col(dial, 'dial')
    
    mm_dataset = trx.join(dial, on='client_id', how='outer').drop(*['trx_src_type21', 'trx_src_type31'])

    mm_dataset.write.mode('overwrite').parquet(os.path.join(MM_DATA_PATH, f'fold={fold}'))
    
    del trx
    del dial
    del mm_dataset

  0%|          | 0/5 [00:00<?, ?it/s]

In [13]:
import pandas as pd
import numpy as np
from ptls.data_load.iterable_processing_dataset import IterableProcessingDataset
from ptls.data_load import IterableChain
from datetime import datetime
from ptls.data_load.datasets.parquet_dataset import ParquetDataset, ParquetFiles
from ptls.data_load.iterable_processing.feature_filter import FeatureFilter
from ptls.data_load.iterable_processing.to_torch_tensor import ToTorch
import torch
from functools import partial
from torch.utils.data import DataLoader
from ptls.data_load.padded_batch import PaddedBatch
from ptls.data_load.utils import collate_feature_dict
from tqdm import tqdm


class TargetToTorch(IterableProcessingDataset):
    def __init__(self, col_target):
        super().__init__()
        self.col_target = col_target

    def __iter__(self):
        for rec in self._src:
            features = rec[0] if type(rec) is tuple else rec
            features[self.col_target] = np.stack(np.array(features[self.col_target]))
            features[self.col_target] = torch.tensor(features[self.col_target])
            yield features

class DeleteNan(IterableProcessingDataset):
    def __init__(self, col_name):
        super().__init__()
        self.col_name = col_name
    
    def __iter__(self):
        for rec in self._src:
            features = rec[0] if type(rec) is tuple else rec
            if features[self.col_name] is not None:
                yield features


class DialToTorch(IterableProcessingDataset):
    def __init__(self, col_time, col_embeds):
        super().__init__()
        self._year=2022
        self.col_embeds = col_embeds
        self.col_time = col_time
    def __iter__(self):
        for rec in self._src:
            features = rec[0] if type(rec) is tuple else rec
            features = features.copy()
            if features[self.col_time] is None:
                features[self.col_time] = torch.tensor([0])
            if features[self.col_embeds] is None:
                features[self.col_embeds] = torch.zeros(768)
            
            for key, tens in features.items():
                if key == self.col_embeds:
                    features[key] = torch.tensor(tens.tolist())

            yield features

class GetSplit(IterableProcessingDataset):
    def __init__(
        self,
        start_month,
        end_month,
        year=2022,
        col_id='client_id',
        col_time='event_time'
    ):
        super().__init__()
        self.start_month = start_month
        self.end_month = end_month
        self._year = year
        self._col_id = col_id
        self._col_time = col_time
        
    def __iter__(self):
        for rec in self._src:
            for month in range(self.start_month, self.end_month+1):
                features = rec[0] if type(rec) is tuple else rec
                features = features.copy()
                
                if month == 12:
                    month_event_time = datetime(self._year + 1, 1, 1).timestamp()
                else:
                    month_event_time = datetime(self._year, month + 1, 1).timestamp()
                    
                year_event_time = datetime(self._year, 1, 1).timestamp()
                
                mask = features[self._col_time] < month_event_time
                
                for key, tensor in features.items():
                    if key.startswith('target'):
                        features[key] = tensor[month - 1].tolist()    
                    elif key != self._col_id:
                        features[key] = tensor[mask] 
                            
                features[self._col_id] += '_month=' + str(month)

                yield features

In [14]:
from ptls.data_load.datasets import ParquetDataset
from ptls.data_load.iterable_processing import SeqLenFilter
from ptls.data_load.iterable_processing.iterable_seq_len_limit import ISeqLenLimit
from ptls.data_load.iterable_processing.to_torch_tensor import ToTorch
train = ParquetDataset(
    data_files=[
        os.path.join(MM_DATA_PATH, f'fold={0}'),
        os.path.join(MM_DATA_PATH, f'fold={1}'),
        os.path.join(MM_DATA_PATH, f'fold={2}')
    ],
    i_filters=[
        DeleteNan('trx_event_time'),
        DeleteNan('dial_event_time'),
        SeqLenFilter(min_seq_len=8),
        ISeqLenLimit(max_seq_len=128),
        ToTorch(),
        DialToTorch(col_time='dial_event_time', col_embeds='dial_embedding'),
        # GetSplit(
        #     start_month=1,
        #     end_month=12,
        #     col_id='client_id'
        # )
    ],
    shuffle_files=True
)
valid = ParquetDataset(
    data_files=[
        os.path.join(MM_DATA_PATH, f'fold={3}')
    ],
    i_filters=[
        DeleteNan('trx_event_time'),
        DeleteNan('dial_event_time'),
        SeqLenFilter(min_seq_len=8),
        ISeqLenLimit(max_seq_len=128),
        ToTorch(),
        DialToTorch(col_time='dial_event_time', col_embeds='dial_embedding'),
        # GetSplit(
        #     start_month=1,
        #     end_month=12,
        #     col_id='client_id'
        # )
    ],
    shuffle_files=False
)

In [15]:
next(iter(train))

<ipython-input-13-35f55d31d8f5>:58: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  features[key] = torch.tensor(tens.tolist())


{'client_id': '000e047a31e50ba35f71c81962b4eb0b9a2d6080cf23a14f91447317dd14c788',
 'trx_event_time': tensor([1653212321, 1653469669, 1653548495, 1653724632, 1653976903, 1654150142,
         1654242596, 1655444259, 1655522300, 1655624414, 1655799111, 1655876123,
         1655882576, 1656039670, 1656138383, 1656223580, 1656596114, 1656605299,
         1656656156, 1656705147, 1656835511, 1656892428, 1656917458, 1656930561,
         1657114146, 1657122681, 1657261902, 1657349967, 1657865226, 1657949365,
         1657988201, 1658211281, 1658817423, 1658964788, 1659005819, 1659316267,
         1659354706, 1659509120, 1659510035, 1659599566, 1660392982, 1660462969,
         1660475549, 1660516728, 1660629900, 1660731542, 1660969768, 1661064107,
         1661065240, 1661074145, 1661143950, 1661590541, 1661668712, 1661856189,
         1662445888, 1662533357, 1662723283, 1662970676, 1663055511, 1663141321,
         1663156346, 1663223349, 1663226110, 1663232930, 1663273613, 1663319977,
         

In [16]:
from ptls.data_load.feature_dict import FeatureDict
from collections import defaultdict
from functools import reduce


class MultiModalDiffSplitDataset(FeatureDict, torch.utils.data.Dataset):
    def __init__(
        self,
        data,
        splitters,
        source_features,
        col_id,
        source_names,
        col_time='event_time',
        *args, **kwargs
    ):
        """
        Dataset for multimodal learning.
        Parameters:
        -----------
        data:
            concatinated data with feature dicts.
        splitter:
            object from from `ptls.frames.coles.split_strategy`.
            Used to split original sequence into subsequences which are samples from one client.
        source_features:
            list of column names 
        col_id:
            column name with user_id
        source_names:
            column name with name sources
        col_time:
            column name with event_time
        """
        super().__init__(*args, **kwargs)
        
        self.data = data
        self.splitters = splitters
        self.col_time = col_time
        self.col_id = col_id
        self.source_names = source_names
        self.source_features = source_features
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        feature_arrays = self.data[idx]
        split_data = self.split_source(feature_arrays)
        # print(self.get_split(split_data))
        return self.get_splits(split_data)
    
    def __iter__(self):
        for feature_arrays in self.data:
            split_data = self.split_source(feature_arrays)
            yield self.get_splits(split_data)
            
    def split_source(self, feature_arrays):
        res = defaultdict(dict)
        for feature_name, feature_array in feature_arrays.items():
            if feature_name == self.col_id:
                res[self.col_id] = feature_array
            else:
                source_name, feature_name_transform = self.get_names(feature_name)
                res[source_name][feature_name_transform] = feature_array
        for source in self.source_names:
            if source not in res:
                res[source] = {source_feature: torch.tensor([]) for source_feature in self.source_features[source]}
        # print(f'res = {res}')
        return res
    
    def get_names(self, feature_name):
        idx_del = feature_name.find('_')
        return feature_name[:idx_del], feature_name[idx_del + 1:]
    
    def get_splits(self, feature_arrays):
        res = {}
        for source_name, feature_array in feature_arrays.items():
            if source_name != self.col_id:
                local_date = feature_array[self.col_time]
                if source_name not in self.splitters:
                    continue
                indexes = self.splitters[source_name].split(local_date)
                res[source_name] = [{k: v[ix] for k, v in feature_array.items() if self.is_seq_feature(k, v)} for ix in indexes]
        return res
    #Вернуть диалоги с транзакциями (без таргетов)
    def collate_fn(self, batch, return_dct_labels=False):
        dict_class_labels = get_dict_class_labels(batch)
        batch = reduce(lambda x, y: {k: x[k] + y[k] for k in x if k in y}, batch)
        padded_batch = collate_multimodal_feature_dict(batch)
        if return_dct_labels:
            return padded_batch, dict_class_labels
        return padded_batch, dict_class_labels[list(dict_class_labels.keys())[0]]

def collate_multimodal_feature_dict(batch):
    res = {}
    for source, source_batch in batch.items():
        res[source] = collate_feature_dict(source_batch)
    # print(f"multimodal_feature_dict = {res['trx'].payload['event_time'].size()}")
    # print()
    return res

def collate_feature_dict(batch):
    new_x_ = defaultdict(list)
    for i, x in enumerate(batch):
        for k, v in x.items():
            new_x_[k].append(v)
    
    seq_col = next(k for k, v in batch[0].items() if FeatureDict.is_seq_feature(k, v))
    lengths = torch.LongTensor([len(rec[seq_col]) for rec in batch])
    new_x = {}
    for k, v in new_x_.items():
        # print(new_x)
        if type(v[0]) is torch.Tensor:
            if k.startswith('target'):
                new_x[k] = torch.stack(v, dim=0)
            else:
                new_x[k] = torch.nn.utils.rnn.pad_sequence(v, batch_first=True)
        elif type(v[0]) is np.ndarray:
            new_x[k] = v  # list of arrays[object]
        else:
            v = np.array(v)
            if v.dtype.kind == 'i':
                new_x[k] = torch.from_numpy(v).long()
            elif v.dtype.kind == 'f':
                new_x[k] = torch.from_numpy(v).float()
            elif v.dtype.kind == 'b':
                new_x[k] = torch.from_numpy(v).bool()
            else:
                new_x[k] = v
    return PaddedBatch(new_x, lengths)
    
def get_dict_class_labels(batch):
    res = defaultdict(list)
    for i, samples in enumerate(batch):
        for source, values in samples.items():
            for _ in values:
                res[source].append(i)
    for source in res:
        res[source] = torch.LongTensor(res[source])
    return dict(res)

class MultiModalDiffSplitIterableDataset(MultiModalDiffSplitDataset, torch.utils.data.IterableDataset):
    pass

In [17]:
from ptls.frames.coles import MultiModalIterableDataset
from lightning.pytorch.loggers import WandbLogger
import ptls

wandb_logger = WandbLogger(project="MBD_My_Code", log_model="all")

trainer = pl.Trainer(
    logger=wandb_logger,
    max_epochs=50,
    accelerator="cuda" if torch.cuda.is_available() else "cpu",
    enable_progress_bar=True,
    gradient_clip_val=0.5,
    log_every_n_steps=50,
    limit_val_batches=36
)

data_module = PtlsDataModule(
    train_data=MultiModalDiffSplitIterableDataset(
        data=train,
        splitters= {
            'trx': SampleSlices(
                split_count=3,
                cnt_min=16,
                cnt_max=90
            ),
            'dial': SampleSlices(
                split_count=3,
                cnt_min=2,
                cnt_max=10
            ),
        },
        source_features={
            "trx": [
                "event_type",
                "event_subtype",
                "src_type11",
                "src_type12",
                "dst_type11",
                "dst_type12",
                "src_type22",
                "src_type32"
            ],
            "dial": [
                "embedding"
            ],
        },
        col_id='client_id',
        col_time='event_time',
        source_names=['trx', 'dial'],
    ),
    valid_data=MultiModalDiffSplitIterableDataset(
        data=valid,
        splitters= {
            'trx': SampleSlices(
                split_count=2,
                cnt_min=5,
                cnt_max=64
            ),
            'dial': SampleSlices(
                split_count=2,
                cnt_min=2,
                cnt_max=10
            ),
            },
        source_features={
            "trx": [
                "event_type",
                "event_subtype",
                "src_type11",
                "src_type12",
                "dst_type11",
                "dst_type12",
                "src_type22",
                "src_type32"
            ],
            "dial": [
                "embedding"
            ],
        },
        col_id='client_id',
        col_time='event_time',
        source_names=['trx', 'dial'],
    ),
    train_batch_size=256,
    train_num_workers=0,
    valid_batch_size=256,
    valid_num_workers=0
)

INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs


In [18]:
from ptls.frames.abs_module import ABSModule
from ptls.frames.coles.metric import metric_recall_top_K, outer_cosine_similarity, outer_pairwise_distance
from ptls.frames.coles.losses import ContrastiveLoss

def first(iterable, default=None):
    iterator = iter(iterable)
    return next(iterator, default)


class M3CoLESModule(ABSModule):
    """
    Multi-Modal Matching
    Contrastive Learning for Event Sequences ([CoLES](https://arxiv.org/abs/2002.08232))

    Subsequences are sampled from original sequence.
    Samples from the same sequence are `positive` examples
    Samples from the different sequences are `negative` examples
    Embeddings for all samples are calculated.
    Paired distances between all embeddings are calculated.
    The loss function tends to make positive distances smaller and negative ones larger.

    Parameters
        seq_encoder:
            Model which calculate embeddings for original raw transaction sequences
            `seq_encoder` is trained by `CoLESModule` to get better representations of input sequences
        head:
            Model which helps to train. Not used during inference
            Can be normalisation layer which make embedding l2 length equals 1
            Can be MLP as `projection head` like in SymCLR framework.
        loss:
            loss object from `ptls.frames.coles.losses`.
            There are paired and triplet loss. They are required sampling strategy
            from `ptls.frames.coles.sampling_strategies`. Sampling strategy takes a relevant pairs or triplets from
            pairwise distance matrix.
        validation_metric:
            Keep None. `ptls.frames.coles.metric.BatchRecallTopK` used by default.
        optimizer_partial:
            optimizer init partial. Network parameters are missed.
        lr_scheduler_partial:
            scheduler init partial. Optimizer are missed.

    """
    def __init__(self,
                 seq_encoders=None,
                 mod_names=None,
                 head=None,
                 loss=None,
                 validation_metric=None,
                 optimizer_partial=None,
                 lr_scheduler_partial=None):
        torch.set_float32_matmul_precision('high')
        if head is None:
            head = ptls.nn.Head(use_norm_encoder=True)

        if loss is None:
            loss = ContrastiveLoss(margin=0.5,
                                   sampling_strategy=HardNegativePairSelector(neg_count=5))

        if validation_metric is None:
            validation_metric = BatchRecallTopK(K=4, metric='cosine')
        
        for k in seq_encoders.keys():
            if type(seq_encoders[k]) is str:
                seq_encoders[k] = seq_encoders[seq_encoders[k]]
                
        super().__init__(validation_metric,
                         first(seq_encoders.values()),
                         loss,
                         optimizer_partial,
                         lr_scheduler_partial)
        
        self.seq_encoders = torch.nn.ModuleDict(seq_encoders)
        self._head = head   
        self.y_h_cache = {'train':[], 'valid': []}
        
    @property
    def metric_name(self):
        return 'recall_top_k'

    @property
    def is_requires_reduced_sequence(self):
        return True
    
    def forward(self, x):
        res = {}
        
        for mod_name in x.keys():
            # print(f'mod_name = {mod_name}')
            res[mod_name] = self.seq_encoders[mod_name](x[mod_name])
        # print(f"forward res = {res['trx'].size()}")
        return res

    def shared_step(self, x, y):
        y_h = self(x)
        
        if self._head is not None:
            y_h_head = {k: self._head(y_h_k) for k, y_h_k in y_h.items()}
            y_h = y_h_head
        # print('share_step........')
        # print(f"y_h = {y_h['trx'].size()}")
        # print(f'y = {y.size()}')
        # print('-----------------------')
        return y_h, y
    
    def _one_step(self, batch, _, stage):
        y_h, y = self.shared_step(*batch)
        y_h_list = list(y_h.values())
        # print("loss count.....")
        # print(f"y_h_list_obj = {y_h_list[0].size()}")
        # print(f"y_true_loss = {torch.cat([y, y]).size()}")
        # print("--------------------------")
        loss = self._loss(torch.cat(y_h_list), torch.cat([y, y]))
        self.log(f'loss/{stage}', loss.detach())
        
        x, y = batch
        # print("step.............")
        # print(f"x_step = {x['trx'].payload['event_time'].size()}")
        # print(f"y_step_true = {y.size()}")
        # print(f"y_h_step = {y_h['trx'].size()}")
        # print("-----------------------")
        for mod_name, mod_x in x.items():
            self.log(f'seq_len/{stage}/{mod_name}', x[mod_name].seq_lens.float().mean().detach(), prog_bar=True)
        
        if stage == "valid":
            n, d = y_h_list[0].shape
            y_h_concat = torch.zeros((2*n, d), device = y_h_list[0].device)
            
            for i in range(2):
                y_h_concat[range(i,2*n,2)] = y_h_list[i] 
            # print('valid.......')
            # print(f'y_h_concat = {y_h_concat[0].size}')
            # print(f"y_h_valid = {y_h['trx'].size()}")
            # print('------------------------')
            if len(self.y_h_cache[stage]) <= 380:
                self.y_h_cache[stage].append((y_h_concat.cpu(), {k: y_h_k.cpu() for k, y_h_k in y_h.items()} , 
                                             {k:x_k.seq_lens.cpu() for k, x_k in x.items()})) 
            # print(self.y_h_cache[stage])
        # print(f'loss = {loss}')
        return loss
    
    def training_step(self, batch, _):
        return self._one_step(batch, _, "train")
    
    def validation_step(self, batch, _):
        return self._one_step(batch, _, "valid")
    
    def on_validation_epoch_end(self):        
        #len_intervals = [(0, 10), (10, 20), (20, 30), (30, 40), (40, 60), (60, 80), (80, 120), (120, 160), (160, 240)]
        self.log_recall_top_K(self.y_h_cache['valid'], len_intervals=None, stage="valid", K=100)
        self.log_recall_top_K(self.y_h_cache['valid'], len_intervals=None, stage="valid", K=50)
        self.log_recall_top_K(self.y_h_cache['valid'], len_intervals=None, stage="valid", K=1)
        
        
        del self.y_h_cache["valid"]
        self.y_h_cache["valid"] = []
        
    def log_recall_top_K(self, y_h_cache, len_intervals=None, stage="valid", K=100):
        y_h = torch.cat([item[0] for item in y_h_cache], dim = 0)
        y_h_mods = defaultdict(list)
        seq_lens_dict = defaultdict(list)
        
        for item in y_h_cache:
            for k, emb in item[1].items():
                y_h_mods[k].append(emb)
                
            for k, l in item[2].items():
                seq_lens_dict[k].append(l)
        
        y_h_mods = {k: torch.cat(el, dim=0) for k ,el in y_h_mods.items()}
        seq_lens_dict = {k: torch.cat(el) for k ,el in seq_lens_dict.items()}

        #n, _ = y_h.shape
        #y = torch.zeros((n,)).cpu().long()
        #y[range(0,n,2)] = torch.arange(0, n//2)
        #y[range(1,n,2)] = torch.arange(0, n//2)
        #computed_metric = metric_real_recall_top_K(y_h, y, K=100)
        y_h_bank, y_h_rmb = list(y_h_mods.values())
        computed_metric_b2r = metric_recall_top_K_for_embs(y_h_bank, y_h_rmb, torch.arange(y_h_rmb.shape[0]), K=K)
        computed_metric_r2b = metric_recall_top_K_for_embs(y_h_rmb, y_h_bank, torch.arange(y_h_rmb.shape[0]), K=K)
        
        if len_intervals != None:
            for mod, seq_lens in seq_lens_dict.items():
                for start, end in len_intervals:
                    mask = ((seq_lens > start) & (seq_lens <= end))

                    if torch.any(mask):
                        #y_h_filtered = y_h[mask.repeat_interleave(2)]
                        y_h_bank_filtered = y_h_bank[mask]
                        y_h_rmb_filtered = y_h_rmb[mask]

                        #y = torch.div(torch.arange(len(y_h_filtered)), 2, rounding_mode='floor')
                        #recall = metric_real_recall_top_K(y_h_filtered, y, K=100)
                        recall_r2b = metric_recall_top_K_for_embs(y_h_rmb_filtered, y_h_bank_filtered, torch.arange(y_h_rmb_filtered.shape[0]), K=100)
                        recall_b2r = metric_recall_top_K_for_embs(y_h_bank_filtered, y_h_rmb_filtered, torch.arange(y_h_rmb_filtered.shape[0]), K=100)

                        #self.log(f"{mode}/R@100_len_from_{start}_to_{end}", recall, prog_bar=True)
                        print(f"{stage}/{mod}/r2b_R@100_len_from_{start}_to_{end}", recall_r2b, prog_bar=True)
                        self.log(f"{stage}/{mod}/b2r_R@100_len_from_{start}_to_{end}", recall_b2r, prog_bar=True)
        
        #self.log(f"{mode}/R@100", computed_metric, prog_bar=True)
        self.log(f"{stage}/click2trx_R@{K}", computed_metric_r2b, prog_bar=True)
        self.log(f"{stage}/trx2click_R@{K}", computed_metric_b2r, prog_bar=True)

In [19]:
from ptls.nn import TrxEncoder, RnnSeqEncoder
from ptls.frames.coles import CoLESModule
from functools import partial
import torch
from ptls.frames.coles import MultiModalSortTimeSeqEncoderContainer
from ptls.nn.trx_encoder.encoders import IdentityEncoder
from ptls.nn.seq_encoder.rnn_encoder import RnnEncoder
from ptls.frames.coles.losses import ContrastiveLoss
from ptls.frames.coles.sampling_strategies import HardNegativePairSelector

head = ptls.nn.Head(
    input_size=128,
    use_norm_encoder=True,
    hidden_layers_sizes=[128, 128],
    objective="regression",
    num_classes=128
)

loss = ptls.frames.coles.losses.SoftmaxLoss()

seq_encoders = {
    'trx': RnnSeqEncoder(
        trx_encoder=TrxEncoder(
            norm_embeddings=False,
            embeddings_noise=0.003,
            linear_projection_size=32,
            embeddings={
                'event_type': {"in": 58, "out": 24},
                'event_subtype': {"in": 59, "out": 24},
                'src_type11': {"in": 85, "out": 24},
                'src_type12': {"in": 349, "out": 24},
                'dst_type11': {"in": 84, "out": 24},
                'dst_type12': {"in": 417, "out": 24},
                'src_type22': {"in": 90, "out": 24},
                'src_type32': {"in": 91, "out": 24}
            },
            numeric_values={
                'amount': 'log'
            }
        ),
        type='gru',
        hidden_size=128
    ),
    'dial': RnnSeqEncoder(
        trx_encoder=TrxEncoder(
            embeddings_noise=0.003,
            linear_projection_size=32,
            custom_embeddings={
                'embedding': IdentityEncoder(768)
            }
        ),
        type='gru',
        hidden_size=128
    )
}

optimizer_partial = partial(
    torch.optim.AdamW,
    lr=0.001,
    weight_decay=1e-4
)

lr_scheduler_partial = partial(
    torch.optim.lr_scheduler.StepLR,
    step_size=1,
    gamma=0.9
)

# pl_module = M3CoLESModule(
#     validation_metric=ptls.frames.coles.metric.BatchRecallTopK(
#         K=1,
#         metric='cosine',
#     ),
#     head=head,
#     seq_encoders=seq_encoders,
#     loss=loss,
#     optimizer_partial=optimizer_partial,
#     lr_scheduler_partial=lr_scheduler_partial
# )

pl_module = M3CoLESModule(
    validation_metric=ptls.frames.coles.metric.BatchRecallTopK(
        K=1,
        metric='cosine',
    ),
    head=None,
    seq_encoders=seq_encoders,
    loss=None,
    optimizer_partial=optimizer_partial,
    lr_scheduler_partial=lr_scheduler_partial
)

In [266]:
pl_module

M3CoLESModule(
  (_loss): ContrastiveLoss()
  (_seq_encoder): RnnSeqEncoder(
    (trx_encoder): TrxEncoder(
      (embeddings): ModuleDict(
        (event_type): NoisyEmbedding(
          58, 24, padding_idx=0
          (dropout): Dropout(p=0, inplace=False)
        )
        (event_subtype): NoisyEmbedding(
          59, 24, padding_idx=0
          (dropout): Dropout(p=0, inplace=False)
        )
        (src_type11): NoisyEmbedding(
          85, 24, padding_idx=0
          (dropout): Dropout(p=0, inplace=False)
        )
        (src_type12): NoisyEmbedding(
          349, 24, padding_idx=0
          (dropout): Dropout(p=0, inplace=False)
        )
        (dst_type11): NoisyEmbedding(
          84, 24, padding_idx=0
          (dropout): Dropout(p=0, inplace=False)
        )
        (dst_type12): NoisyEmbedding(
          417, 24, padding_idx=0
          (dropout): Dropout(p=0, inplace=False)
        )
        (src_type22): NoisyEmbedding(
          90, 24, padding_idx=0
          (

In [20]:
import warnings
warnings.filterwarnings("ignore")

In [21]:
def metric_real_recall_top_K(X, y, K, num_pos=1, metric='cosine'):
    """
        calculate metric R@K
        X - tensor with size n x d, where n - number of examples, d - size of embedding vectors
        y - true labels
        N - count of closest examples, which we consider for recall calcualtion
        metric: 'cosine' / 'euclidean'.
            !!! 'euclidean' - to slow for datasets bigger than 100K rows
    """
    K_adjusted = min(X.size(0) - 1, K)
    
    res = []

    n = X.size(0)
    d = X.size(1)
    max_size = 2 ** 32
    batch_size = max(1, max_size // (n * d))

    with torch.no_grad():

        for i in range(1 + (len(X) - 1) // batch_size):

            id_left = i * batch_size
            id_right = min((i + 1) * batch_size, len(y))
            y_batch = y[id_left:id_right]
            print(f"X = {X}")
            print(f"X[] = {X[id_left:id_right]}")
            if metric == 'cosine':
                pdist = -1 * outer_cosine_similarity(X, X[id_left:id_right])
            elif metric == 'euclidean':
                pdist = outer_pairwise_distance(X, X[id_left:id_right])
            else:
                raise AttributeError(f'wrong metric "{metric}"')

            values, indices = pdist.topk(K_adjusted + 1, 0, largest=False)

            y_rep = y_batch.repeat(K_adjusted, 1)
            res.append((y[indices[1:]] == y_rep).sum().item())

    return np.sum(res) / len(y) / num_pos

def cosine_similarity_matrix(x1, x2):
    x1_norm = x1 / x1.norm(dim=1)[:, None]
    x2_norm = x2 / x2.norm(dim=1)[:, None]
    return torch.mm(x1_norm, x2_norm.transpose(0, 1))

def metric_recall_top_K_for_embs(embs_1, embs_2, true_matches, K=100):
    similarity_matrix = cosine_similarity_matrix(embs_1, embs_2)
    K_adjusted = min(len(embs_1), K)
    top_k = similarity_matrix.topk(k=K_adjusted, dim=1).indices
    correct_matches = 0
    for i, indices in enumerate(top_k):
        if true_matches[i] in indices:
            correct_matches += 1
    recall_at_k = correct_matches / len(similarity_matrix)
    return recall_at_k


In [26]:
trainer.fit(pl_module, data_module)

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: 
Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [ ]:
print(trainer.logged_metrics)

In [ ]:
torch.save(pl_module.state_dict(), "embedder.pt")

# Downstream Task

**Get embeddings for last fold**

**variant 1**

In [27]:
inference_dataset = ParquetDataset(
    data_files=[
        os.path.join(MM_DATA_PATH, f'fold={4}')
    ],
    i_filters=[
        DeleteNan('trx_event_time'),
        DeleteNan('dial_event_time'),
        SeqLenFilter(min_seq_len=1),
        ISeqLenLimit(max_seq_len=128),
        ToTorch(),
        DialToTorch(col_time='dial_event_time', col_embeds='dial_embedding')
    ],
    shuffle_files=False
)

inference_dataset = MultiModalDiffSplitIterableDataset(
        data=inference_dataset,
        splitters= {
            'trx': ptls.frames.coles.split_strategy.NoSplit(),
            'dial': ptls.frames.coles.split_strategy.NoSplit(),
        },
        source_features={
            "trx": [
                "event_type",
                "event_subtype",
                "src_type11",
                "src_type12",
                "dst_type11",
                "dst_type12",
                "src_type22",
                "src_type32",
                "event_time"
            ],
            "dial": [
                "embedding",
                "event_time"
            ],
        },
        col_id='client_id',
        col_time='event_time',
        source_names=['trx', 'dial'],
    )

In [28]:
def collate_fn_multimodal(batch):
    dict_class_labels = get_dict_class_labels(batch)
    batch = reduce(lambda x, y: {k: x[k] + y[k] for k in x if k in y}, batch)
    padded_batch = collate_multimodal_feature_dict(batch)
    return padded_batch

In [29]:
from torch.utils.data import DataLoader
embeddings_dataloader = DataLoader(
    inference_dataset, 
    collate_fn=partial(collate_fn_multimodal),
    shuffle=False,
    num_workers=0,
    batch_size=256
)

In [30]:
next(iter(embeddings_dataloader))

{'trx': <ptls.data_load.padded_batch.PaddedBatch at 0x7acfa978b640>,
 'dial': <ptls.data_load.padded_batch.PaddedBatch at 0x7acfa97891e0>}

In [ ]:
inference_embeddings = trainer.predict(pl_module, embeddings_dataloader)

In [ ]:
inference_embeddings[0]

**variant 2**

# Getting targets

In [22]:
TARGETS_DATA_PATH = '/kaggle/working/targets/'

preprocessor_target = PysparkDataPreprocessor(
    col_id="client_id",
    col_event_time="mon",
    event_time_transformation="dt_to_timestamp",
    cols_identity=["target_1", "target_2", "target_3", "target_4"],
)


In [23]:
targets = spark.read.parquet(os.path.join(TARGETS_DATA_PATH , f'fold={fold}'))
mmt_dataset = spark.read.parquet(os.path.join(MM_DATA_PATH , f'fold={fold}'))

targets = preprocessor_target.fit_transform(targets).drop(*['event_time' ,'trans_count', 'diff_trans_date'])
mmt_dataset = mmt_dataset.join(targets, on='client_id', how='left')
mmt_dataset.write.mode('overwrite').parquet(os.path.join(MMT_DATA_PATH, f'fold={fold}'))

del mmt_dataset

In [498]:
class MatchingModalities(IterableProcessingDataset):
    def __init__(
        self,
        col_id='client_id',
        mod1_col_time='trx_event_time',
        mod2_col_time='dial_event_time',
        mod1_name='trx',
        mod2_name='dial'
    ):
        super().__init__()
        self.col_id = col_id
        self.mod1_col_time = mod1_col_time
        self.mod2_col_time = mod2_col_time
        self.mod1_name=mod1_name
        self.mod2_name=mod2_name

    def __iter__(self):
        for rec in self._src:
            features = rec[0] if type(rec) is tuple else rec
            features = features.copy()
            all_event_time, _ = torch.sort(torch.cat((features[self.mod1_col_time], features[self.mod2_col_time])))
            mod1_mask = torch.isin(all_event_time, features[self.mod1_col_time])
            mod2_mask = torch.isin(all_event_time, features[self.mod2_col_time])
            for key, tens in features.items():
                if key.startswith(self.mod1_name) and key != self.mod1_col_time:
                    indices = torch.where(~mod1_mask)[0].tolist()
                    result = []
                    for i in range(len(mod1_mask)):
                        if i in indices:
                            result.append(0)
                        if i < len(tens):
                            result.append(tens[i])
                    features[key] = torch.tensor(result, dtype=features[key].dtype)
                    # print(f'mod1_size = {features[key].size()}')
                elif key.startswith(self.mod2_name) and key != self.mod2_col_time:
                    indices = torch.where(~mod2_mask)[0].tolist()
                    # print(indices)
                    result = []
                    for i in range(len(mod2_mask)):
                        if i in indices:
                            result.append(torch.Tensor([0 for i in range(768)]))
                        if i < len(tens):
                            result.append(tens[i])
                    # print(len(result))
                    # print(result)
                    features[key] = torch.stack(result, dim=0)
                    # print(f'mod2_size = {features[key].size()}')
            features[self.mod1_col_time] = all_event_time
            features[self.mod2_col_time] = all_event_time
            
            yield features   

class MMToTorch(IterableProcessingDataset):
    def __init__(
        self,
        col_id='client_id'
    ):
        super().__init__()
        self.col_id='client_id'

    def __iter__(self):
        for rec in self._src:
            features = rec[0] if type(rec) is tuple else rec
            features = features.copy()
            for key, value in features.items():
                if key != 'client_id':
                    features[key] = torch.Tensor(value)
            yield features

class DialToTorch(IterableProcessingDataset):
    def __init__(
        self,
        embedding_col='dial_embedding'
    ):
        super().__init__()
        self.embedding_col=embedding_col

    def __iter__(self):
        for rec in self._src:
            features = rec[0] if type(rec) is tuple else rec
            features = features.copy()
            features[self.embedding_col] = np.stack(features[self.embedding_col], axis=0).astype(np.int32)
            features[self.embedding_col] = torch.FloatTensor(features[self.embedding_col])
            yield features

In [499]:
class GetSplit(IterableProcessingDataset):
    def __init__(
        self,
        start_month,
        end_month,
        year=2022,
        col_id='client_id',
        col_time='event_time'
    ):
        super().__init__()
        self.start_month = start_month
        self.end_month = end_month
        self._year = year
        self._col_id = col_id
        self._col_time = col_time
        
    def __iter__(self):
        for rec in self._src:
            for month in range(self.start_month, self.end_month+1):
                features = rec[0] if type(rec) is tuple else rec
                features = features.copy()
                
                if month == 12:
                    month_event_time = datetime(self._year + 1, 1, 1).timestamp()
                else:
                    month_event_time = datetime(self._year, month + 1, 1).timestamp()
                    
                year_event_time = datetime(self._year, 1, 1).timestamp()
                
                mask = features[self._col_time] < month_event_time
                
                for key, tensor in features.items():
                    if key.startswith('target'):
                        features[key] = tensor[month - 1].tolist()    
                    elif key != self._col_id:
                        features[key] = tensor[mask] 
                            
                features[self._col_id] += '_month=' + str(month)

                yield features

In [500]:
from ptls.frames.coles import MultiModalInferenceIterableDataset

dataset_inf = ParquetDataset(
    data_files=[
        os.path.join(MMT_DATA_PATH, f'fold={4}')
    ],
    i_filters=[
        DeleteNan('trx_event_time'),
        DeleteNan('dial_event_time'),
        # SeqLenFilter(min_seq_len=1),
        # ISeqLenLimit(max_seq_len=128),
        ptls.data_load.iterable_processing.to_torch_tensor.ToTorch(),
        # MMToTorch('client_id'),
        DialToTorch(),
        MatchingModalities(
            col_id='client_id',
            mod1_col_time='trx_event_time',
            mod2_col_time='dial_event_time'
        ),
        GetSplit(
            start_month=1,
            end_month=12,
            col_id='client_id',
            col_time='trx_event_time'
        )
    ],
    shuffle_files=False
)

dataset_inf = MultiModalInferenceIterableDataset(
        data=dataset_inf,
        source_features={
            "trx": [
                "event_type",
                "event_subtype",
                "src_type11",
                "src_type12",
                "dst_type11",
                "dst_type12",
                "src_type22",
                "src_type32"
            ],
            "dial": [
                "embedding"
            ],
        },
        col_id='client_id',
        col_time='event_time',
        source_names=['trx', 'dial'],
    )

In [501]:
next(iter(dataset_inf))

{'client_id': ['001755891452af8d541d0466d6995b768904e181a74a65e0adfa7bc45c57475d_month=1'],
 'trx': [{'event_time': tensor([1611936689, 1612018802, 1612832521, 1613587457, 1613669486, 1614969839,
           1615293738, 1616354267, 1616585402, 1617910706, 1617995463, 1619289462,
           1619318787, 1619437601, 1621195367, 1621943545, 1622690253, 1622748925,
           1623197504, 1623491185, 1623717871, 1624739375, 1625043408, 1625479774,
           1628078533, 1628264749, 1629393621, 1629914943, 1630241533, 1630960006,
           1631639042, 1631820719, 1631867063, 1633392592, 1633424276, 1633581407,
           1633904143, 1634178802, 1634222704, 1634402064, 1634402754, 1634716867,
           1634750013, 1635441875, 1637454702, 1637952046, 1638550048, 1639297932,
           1639639847, 1640266108, 1640626342, 1642272787, 1642360376]),
   'amount': tensor([5.9647e+02, 3.1160e+05, 6.2832e+02, 1.3094e+02, 2.3825e+02, 3.9268e+02,
           1.9584e+05, 6.8911e+01, 2.5653e+05, 2.3097e+01

In [525]:
cntr = 0
for item in dataset_inf:
    if cntr > 80 and cntr < 82:
        print(item)
        print('---------------------------')
    if cntr > 82:
        break
    cntr += 1

{'client_id': ['00720bf69eb71320337ae0102830206e21eb205482898b8021a05fa4da4913bd_month=10'], 'trx': [{'event_time': tensor([1609582021, 1609584439, 1609610403,  ..., 1667210098, 1667224811,
        1667230486]), 'amount': tensor([3.6500e+04, 2.0009e+06, 9.2271e+02,  ..., 2.3276e+04, 7.3116e+00,
        3.0420e+04]), 'event_type': tensor([ 1,  1,  3,  ...,  1, 18, 11], dtype=torch.int32), 'event_subtype': tensor([ 2,  1,  3,  ...,  2, 19, 12], dtype=torch.int32), 'currency': tensor([1, 1, 1,  ..., 1, 1, 1], dtype=torch.int32), 'src_type11': tensor([1, 1, 1,  ..., 1, 1, 1], dtype=torch.int32), 'src_type12': tensor([1, 1, 1,  ..., 1, 1, 1], dtype=torch.int32), 'dst_type11': tensor([ 2,  1,  3,  ...,  2, 12,  1], dtype=torch.int32), 'dst_type12': tensor([20,  1,  3,  ..., 20, 16,  1], dtype=torch.int32), 'src_type22': tensor([1, 1, 1,  ..., 1, 1, 1], dtype=torch.int32), 'src_type32': tensor([16, 16, 16,  ..., 16, 16, 16], dtype=torch.int32)}], 'dial': [{'event_time': tensor([1609582021, 16

In [512]:
# from ptls.custom_layers import StatPooling
# from ptls.nn.seq_step import LastStepEncoder
from itertools import chain

class InferenceModuleMultimodal(pl.LightningModule):
    def __init__(self, model, pandas_output=True, drop_seq_features=True, model_out_name='out', col_id = 'epk_id'):
        super().__init__()

        self.model = model
        self.pandas_output = pandas_output
        self.drop_seq_features = drop_seq_features
        self.model_out_name = model_out_name
        self.col_id = col_id

    def forward(self, x: PaddedBatch):
        x, batch_ids, _ = x

        #TODO: Проверить batch_ids == col_id. Проверил, совпадают
        out = self.model(x)
        x_out = {self.col_id : batch_ids, self.model_out_name: out}
        if self.pandas_output:
            return self.to_pandas(x_out)
        return x_out

    def to_pandas(self, x):
        expand_cols = []
        scalar_features = {}
        if self.model_out_name in x:
            for k, v in x[self.model_out_name].items():
                x[k] = v
        del x[self.model_out_name]
        for k, v in x.items():
            if type(v) is torch.Tensor:
                v = v.cpu().numpy()
            if type(v) is list or len(v.shape) == 1:
                scalar_features[k] = v
            elif len(v.shape) == 2:
                expand_cols.append(k)
            else:
                scalar_features[k] = None

        dataframes = [pd.DataFrame(scalar_features)]
        for col in expand_cols:
            v = x[col].cpu().numpy()
            dataframes.append(pd.DataFrame(v, columns=[f'{col}_{i:04d}' for i in range(v.shape[1])]))

        return pd.concat(dataframes, axis=1)

In [582]:
def collate_multimodal_feature_dict(batch):
    res = {}
    for source, source_batch in batch[0].items():
        # print(f'source = {source}')
        # print(f'source_batch = {source_batch}')
        # print(f'source_batch = {source_batch}')
        if source_batch[0]['event_time'].size() == (1, 0):
            for key, value in source_batch[0].items():
                if key.startswith('dial_emb'):
                    source_batch[0][key] = torch.zeros((1, 768))
                elif key.startswith('trx') or key.startswith('dial'):
                    source_batch[0][key] = torch.zeros(1)
        res[source] = collate_feature_dict(source_batch)
    # print(f'res = {res}')
    # print(f'res_dial = {res["dial"].payload}')
    return res

def collate_feature_dict_with_target(batch, col_id='client_id', target_col_names=None):
    batch_ids = []
    target_cols = []
    for sample in batch:
        batch_ids.append(sample[col_id])
        del sample[col_id]
        if target_col_names is not None:
            for target_col in target_col_names:
                target_cols.append(sample[target_col])
                del sample[target_col]
    padded_batch = collate_multimodal_feature_dict(batch)
    if target_col_names is not None:
        return padded_batch, batch_ids, target_cols
    return padded_batch, batch_ids[0]

In [583]:
from torch.utils.data import DataLoader

collate_fn = partial(
    collate_feature_dict_with_target,
    target_col_names=['target']
)

inference_dl = DataLoader(
    dataset=dataset_inf,
    collate_fn=collate_fn,
    shuffle=False,
    num_workers=0,
    batch_size=2
)

In [584]:
# a = 0
# next(iter(inference_dl))
# b = 2

In [585]:
inf_module = InferenceModuleMultimodal(
    model=pl_module,
    pandas_output=True,
    col_id='client_id',
)

In [586]:
cntr = 0
for item in inference_dl:
    if cntr == 162:
        print(item[0])
        print(item[0]['dial'].payload)
        print('---------------------------')
    if cntr > 200:
        break
    cntr += 1

{'trx': <ptls.data_load.padded_batch.PaddedBatch object at 0x7ba28db5b880>, 'dial': <ptls.data_load.padded_batch.PaddedBatch object at 0x7ba28db580a0>}
{'event_time': tensor([], size=(1, 0), dtype=torch.int64), 'embedding': tensor([], size=(1, 0, 768))}
---------------------------


In [580]:
cntr = 0
for item in inference_dl:
    if cntr > 80 and cntr < 82:
        print(item[0])
        print(item[0]['trx'].payload)
        print('---------------------------')
    if cntr > 82:
        break
    cntr += 1

{'trx': <ptls.data_load.padded_batch.PaddedBatch object at 0x7ba28db5ace0>, 'dial': <ptls.data_load.padded_batch.PaddedBatch object at 0x7ba28db5b880>}
{'event_time': tensor([[1609472973, 1609728676, 1609818706, 1610101269, 1610874609, 1610937003,
         1611382291, 1611483166, 1611631739, 1611676093, 1611725759, 1611804497,
         1612015594, 1612272731, 1612416875, 1612626863, 1612631744, 1613440275,
         1613793421, 1614043273, 1614064401, 1614157487, 1614305304, 1614508599,
         1614580173, 1614684499, 1614759315, 1614783413, 1614823600, 1615002125,
         1616210133, 1616583427, 1616667546, 1616855910, 1616896249, 1616921102,
         1617095354, 1617124791, 1617245557, 1617614703, 1617620793, 1617720347,
         1617845307, 1617875986, 1618392354, 1618496647, 1618633599, 1619239094,
         1619593738, 1620193338, 1620274954, 1620539611, 1620796405, 1621131392,
         1621745137, 1621931251, 1622013754, 1622177303, 1622182215, 1622186992,
         1622685050, 16

In [557]:
inf_embeddings = pd.concat(trainer.predict(inf_module, inference_dl))

Predicting: |          | 0/? [00:00<?, ?it/s]

AssertionError: Batch can'not have 0 transactions

In [556]:
inf_embeddings

,client_id,trx_0000,trx_0001,trx_0002,trx_0003,trx_0004,trx_0005,trx_0006,trx_0007,trx_0008,...,dial_0118,dial_0119,dial_0120,dial_0121,dial_0122,dial_0123,dial_0124,dial_0125,dial_0126,dial_0127
0,[001755891452af8d541d0466d6995b768904e181a74a6...,0.008138,-0.381437,-0.173643,0.040975,0.350735,-0.012193,-0.084212,-0.249949,-0.015709,...,0.046680,-0.019130,0.021606,0.069314,-0.032143,0.068159,0.106310,-0.025588,0.024027,0.075063
1,[001755891452af8d541d0466d6995b768904e181a74a6...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[001755891452af8d541d0466d6995b768904e181a74a6...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[001755891452af8d541d0466d6995b768904e181a74a6...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,[001755891452af8d541d0466d6995b768904e181a74a6...,0.005664,-0.386670,-0.143982,0.031381,0.353962,-0.005376,-0.096481,-0.246804,-0.020569,...,0.046680,-0.019130,0.021606,0.069314,-0.032143,0.068159,0.106310,-0.025588,0.024027,0.075063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,[0141b8fc69c73eae094415c85df4fc3bac87177b0a8d3...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,[0141b8fc69c73eae094415c85df4fc3bac87177b0a8d3...,-0.135283,-0.453365,-0.143856,-0.076178,0.477253,0.147498,-0.073590,-0.125378,-0.055532,...,0.046681,-0.019129,0.021605,0.069314,-0.032143,0.068159,0.106309,-0.025589,0.024028,0.075064
1,[0141b8fc69c73eae094415c85df4fc3bac87177b0a8d3...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[0141b8fc69c73eae094415c85df4fc3bac87177b0a8d3...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
inf_embeddings

**join targets with mm_dataset for test**

In [ ]:
import random

def drop_zeros_targets(df, drop_frequency=1):
    cols_to_drop = []
    for i in range(len(df)):
        if len(np.unique(df['target'][i])) == 1 and random.random() < drop_frequency:
            cols_to_drop.append(i)
    return df.drop(cols_to_drop).reset_index(drop=True)


In [ ]:
# mmt_dataset = drop_zeros_targets(mmt_dataset)

In [ ]:
mmt_dataset

In [ ]:
from catboost import Pool, CatBoostClassifier
from sklearn.model_selection import train_test_split

In [ ]:
inf_train, inf_test = train_test_split(mmt_dataset, test_size=0.2)
inf_train, inf_val = train_test_split(inf_train, test_size=0.1)

In [ ]:
y_inf_train, y_inf_test = inf_train['target'].to_numpy(), inf_test['target'].to_numpy()
X_inf_train, X_inf_test = inf_train.drop(columns=['client_id', 'target']).to_numpy(), inf_test.drop(columns=['client_id', 'target']).to_numpy()
X_inf_val, y_inf_val = inf_val.drop(columns=['client_id', 'target']).to_numpy(), inf_val['target'].to_numpy()

In [ ]:
y_inf_train = np.vstack(y_inf_train)
y_inf_test = np.vstack(y_inf_test)
y_inf_val = np.vstack(y_inf_val)

In [ ]:
y_inf_train.shape

**Use nn head**

In [ ]:
from torch.utils.data import TensorDataset
import torch.nn as nn
from sklearn.metrics import classification_report

class LightningModel(pl.LightningModule):
    def __init__(self, input_size=256, hidden_size=64, output_size=48, lr=0.001):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_size, output_size)
        )
        self.criterion = nn.BCEWithLogitsLoss()
        self.lr = lr

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y = y.float().to("cuda:0")
        logits = self(x).type(torch.FloatTensor).to(self.device)
        loss = self.criterion(logits, y.float())
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)

    def predict_step(self, batch, batch_idx):
        x, _ = batch
        preds = torch.sigmoid(self(x))
        return preds

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x).type(torch.FloatTensor).to(self.device)
        loss = self.criterion(logits, y.float())
        self.log('val_loss', loss, prog_bar=True)

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x).type(torch.FloatTensor).to(self.device)
        loss = self.criterion(logits, y.float())
        preds = (torch.sigmoid(logits) > 0.5).float()
        acc = (preds == y).float().mean()
        self.log('test_acc', acc)

In [ ]:
X_inf_train_tensor = torch.from_numpy(X_inf_train)
y_inf_train_tensor = torch.from_numpy(y_inf_train)

X_inf_val_tensor = torch.from_numpy(X_inf_val)
y_inf_val_tensor = torch.from_numpy(y_inf_val)

X_inf_test_tensor = torch.from_numpy(X_inf_test)
y_inf_test_tensor = torch.from_numpy(y_inf_test)

In [ ]:
train_set = TensorDataset(X_inf_train_tensor, y_inf_train_tensor)
val_set = TensorDataset(X_inf_val_tensor, y_inf_val_tensor)

In [ ]:
model = LightningModel()
trainer = pl.Trainer(
    max_epochs=15,
    accelerator="cuda",
    enable_progress_bar=True
)
trainer.fit(model, train_set, val_set)

In [ ]:
model.eval()
test_set = TensorDataset(X_inf_test_tensor, y_inf_test_tensor)
with torch.no_grad():
    trainer.test(model, test_set)

**Use catboost**

In [ ]:
# y_inf_train = y_inf_train.transpose()
# y_inf_test = y_inf_test.transpose()
# y_inf_val = y_inf_val.transpose()

In [ ]:
np.unique(y_inf_train, return_counts=True)

In [ ]:
train_dataset = Pool(X_inf_train, y_inf_train)
val_dataset = Pool(X_inf_val, y_inf_val)

In [ ]:
model = CatBoostClassifier(
    iterations=500,
    max_depth=6,
    learning_rate=0.02,
    task_type='GPU',
    devices='0',
    loss_function='MultiCrossEntropy',
    verbose=False
)

In [ ]:
model.fit(train_dataset, eval_set=val_dataset)

In [ ]:
preds = model.predict(X_inf_test)

In [ ]:
np.unique(y_inf_test, return_counts=True)

In [ ]:
np.unique(preds, return_counts=True)

In [ ]:
np.unique(preds)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_inf_test, preds))